# Imp Catcher
## A Dark Chess Engine and Open Research Project

### Goals:
- Build a state of the art Dark Chess Engine
- Learn about Game theoertic concepts for Deeply complex and large games
- Intuit how agents can scale to very large models
- Build out mechanisms to efficiently and scalably compete agents for Dark Chess


### Tools
- The Julia Language
- anoojpatel/Chess.jl fork to implement Dark Chess
- anooj/ImpCatcher repo to implement algorithms and gyms

### Install Julia into the machine and instantiate the kernel for use.

In [ ]:
# Installation cell
!wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
    -O /tmp/julia.tar.gz
!tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
!rm /tmp/julia.tar.gz

!julia -e 'using Pkg; pkg"add IJulia; precompile;"'
!echo 'Done'
     

**Note, you'll need to restart the kernel & choose the Julia Kernel**

In [20]:
# Test GPU access
using Pkg
Pkg.add(["BenchmarkTools", "CUDA"])
using BenchmarkTools, CUDA

if has_cuda_gpu()
  print("The GPU device is:", CUDA.device())
end

CUDA.device()

The GPU device is:CuDevice(0)

   Resolving package versions...
    Updating `/notebooks/ImpCatcher/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [052768ef] + CUDA v3.13.1
    Updating `/notebooks/ImpCatcher/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [9abbd945] + Profile


CuDevice(0): Quadro M4000

### Activating the Environment

#### Let's activate the Environment from the ImpCatcher package. Note, in julia, we have a similiar Package manager to Rust called `Pkg` ([Check out the docs!](https://pkgdocs.julialang.org/v1/getting-started/)) 
> `Manifest.toml` & `Project.toml` is where the state of our packages is managed. Pkg resolves high level dependency asks (in Project.toml) and resolves into into a determinstic state (in Manifest.toml)


In [2]:
using Pkg
Pkg.activate("ImpCatcher")
Pkg.precompile()
Pkg.instantiate()

  Activating project at `/notebooks/ImpCatcher`
     Cloning git-repo `https://github.com/WhiffleFish/ESCHER.jl.git`
     Cloning git-repo `https://github.com/anoojpatel/Chess.jl.git`
   Installed Calculus ────────────────────────── v0.5.1
   Installed ShowCases ───────────────────────── v0.1.0
   Installed Accessors ───────────────────────── v0.1.25
   Installed StatsFuns ───────────────────────── v1.1.1
   Installed ContextVariablesX ───────────────── v0.1.3
   Installed NNlib ───────────────────────────── v0.8.16
   Installed Unitful ─────────────────────────── v1.12.2
   Installed PDMats ──────────────────────────── v0.11.16
   Installed Contour ─────────────────────────── v0.5.7
   Installed HTML_Entities ───────────────────── v1.0.1
   Installed NNlibCUDA ───────────────────────── v0.2.6
   Installed JuliaFormatter ──────────────────── v1.0.20
   Installed TimerOutputs ────────────────────── v0.5.22
   Installed MarchingCubes ───────────────────── v0.1.6
   Installed InlineString

     Cloning git-repo `https://github.com/WhiffleFish/ESCHER.jl.git`
    Updating git-repo `https://github.com/WhiffleFish/ESCHER.jl.git`
   Resolving package versions...
    Updating `/notebooks/ImpCatcher/Project.toml`
  [8352d375] ~ ESCHER v0.1.0 `~/.julia/dev/ESCHER` ⇒ v0.1.0 `https://github.com/WhiffleFish/ESCHER.jl.git#main`
    Updating `/notebooks/ImpCatcher/Manifest.toml`
  [8352d375] ~ ESCHER v0.1.0 `~/.julia/dev/ESCHER` ⇒ v0.1.0 `https://github.com/WhiffleFish/ESCHER.jl.git#main`
Precompiling project...
  ✓ ESCHER
  ✓ ImpCatcher
  2 dependencies successfully precompiled in 10 seconds (194 already precompiled)


In [25]:
Pkg.add("Flux")

   Resolving package versions...
    Updating `/notebooks/ImpCatcher/Project.toml`
  [587475ba] + Flux v0.13.4
  No Changes to `/notebooks/ImpCatcher/Manifest.toml`


___

## Understanding our Bot
#### Table of Contents
**1. Background in Solving Dark Chess**  

**2. High Level Architecture** 

**3. Low Level Implementation** 

**4. Training** 

**5. The ELI5 version of the Math Concepts**  

----
### WIP Explaination ####

# Model building

In [3]:
using Transformers
using Transformers.Basic
using Flux
N = 3
encoder = Stack(
    @nntopo(e → pe:(e, pe) → x → x → $N),
    PositionEmbedding(512),
    (e, pe) -> e .+ pe,
    Dropout(0.1),
    [Transformer(512, 8, 64, 2048) for i = 1:N]...
)


[ Info: Precompiling Transformers [21ca0261-441d-5938-ace7-c90938fde4d4]


Stack(PositionEmbedding(512), #1, Dropout(0.1), Transformer(head=8, head_size=64, pwffn_size=2048, size=512), Transformer(head=8, head_size=64, pwffn_size=2048, size=512), Transformer(head=8, head_size=64, pwffn_size=2048, size=512))

In [4]:
avg_policy_decoder = Stack(
    @nntopo((e, m, mask):e → pe:(e, pe) → t → (t:(t, m, mask) → t:(t, m, mask)) → $N:t → c → c),
    PositionEmbedding(512),
    (e, pe) -> e .+ pe,
    Dropout(0.1),
    [TransformerDecoder(512, 8, 64, 2048) for i = 1:N]...,
    Positionwise(Dense(512, 4096), logsoftmax)
)

Stack(PositionEmbedding(512), #5, Dropout(0.1), TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512), TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512), TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512), Positionwise(Dense(512 => 4096), logsoftmax))

In [12]:
using OpenSpiel

LoadError: ArgumentError: Package OpenSpiel not found in current path:
- Run `import Pkg; Pkg.add("OpenSpiel")` to install the OpenSpiel package.


In [3]:
using ReinforcementLearning


In [2]:
using OpenSpiel


In [ ]:
sleep(5)
OpenSpielEnv("dark_chess")

In [7]:
OpenSpiel.registered_names()[20:30]

11-element Vector{CxxWrap.StdLib.StdString}:
 "cursor_go"
 "dark_chess"
 "dark_hex"
 "dark_hex_ir"
 "deep_sea"
 "efg_game"
 "first_sealed_auction"
 "gin_rummy"
 "go"
 "goofspiel"
 "hanabi"

### Genereate the transformations from 